# Ultralytics YOLO11 L3 Base Detector Training

**Objective:** Train a robust, high-performance **YOLO11n** detector for the fine-grained (L3 leaf) vehicle classes. This notebook serves as the first crucial stage in our decoupled training strategy, leveraging the latest state-of-the-art architecture.

**Core Principles of this Notebook:**
- **Efficiency:** We will use caching, optimized dataloaders, and mixed-precision training.
- **Reproducibility:** All hyperparameters are centralized in a single `hyp.yaml` file, created and archived with each run.
- **Effectiveness:** We will tackle the dataset's severe class imbalance head-on using state-of-the-art loss weighting (`effective-number`) and augmentations.

The final output of this notebook will be the `best.pt` model, ready for the next stage: hard-negative mining and auxiliary head training.

In [1]:
import os
import random
import json
import platform
from pathlib import Path
from datetime import datetime

# Core ML/CV libraries
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import yaml

# Check for ultralytics and provide a helpful message if not found
try:
    import ultralytics
    print(f"Ultralytics version: {ultralytics.__version__}")

    # Disable Ultralytics AMP download check (which otherwise downloads YOLOv8n)
    try:
        from ultralytics.utils import checks as _ultra_checks
        def _skip_amp_check(*args, **kwargs):
            # Assume AMP is OK; skip model-based probe that downloads yolov8n
            return True
        _ultra_checks.check_amp = _skip_amp_check
        print("Ultralytics checks.check_amp disabled to prevent YOLOv8n download.")
    except Exception as _e:
        print(f"Could not disable ultralytics.utils.checks.check_amp: {_e}")

    # Some versions use ultralytics.utils.torch_utils.check_amp for the probe
    try:
        from ultralytics.utils import torch_utils as _ultra_tu
        def _skip_amp_check_tu(*args, **kwargs):
            # Assume AMP is OK; skip model-based probe that downloads yolov8n
            return True
        _ultra_tu.check_amp = _skip_amp_check_tu
        print("Ultralytics torch_utils.check_amp disabled to prevent YOLOv8n download.")
    except Exception as _e:
        print(f"Could not disable ultralytics.utils.torch_utils.check_amp: {_e}")

    # Some modules import check_amp by name; patch trainer bindings too
    try:
        from ultralytics.engine import trainer as _ultra_trainer
        if hasattr(_ultra_trainer, 'check_amp'):
            _ultra_trainer.check_amp = _skip_amp_check_tu
        # Also patch Trainer class if method exists (static or class method)
        if hasattr(_ultra_trainer, 'Trainer') and hasattr(_ultra_trainer.Trainer, 'check_amp'):
            try:
                # If it's a @staticmethod
                _ultra_trainer.Trainer.check_amp = staticmethod(_skip_amp_check_tu)
            except Exception:
                try:
                    # Fallback assignment
                    _ultra_trainer.Trainer.check_amp = _skip_amp_check_tu
                except Exception:
                    pass
        print("Ultralytics trainer-level AMP checks disabled.")
    except Exception as _e:
        print(f"Could not patch trainer-level AMP check: {_e}")
except ImportError:
    raise ImportError("Ultralytics not found. Please install with: pip install ultralytics")

def set_all_seeds(seed: int = 2, deterministic: bool = False):
    """Sets random seeds for reproducibility across libraries."""
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        if deterministic:
            cudnn.deterministic = True
            cudnn.benchmark = False
        else:
            # Faster but less reproducible
            cudnn.deterministic = False
            cudnn.benchmark = True
    print(f"Seeds set to {seed}. Deterministic: {deterministic}")

# Environment Snapshot
ENV = {
    "timestamp": datetime.utcnow().isoformat() + "Z",
    "python": platform.python_version(),
    "platform": platform.platform(),
    "torch": torch.__version__,
    "cuda_available": torch.cuda.is_available(),
    "gpu": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "N/A",
}
print("\n--- Environment ---")
print(json.dumps(ENV, indent=2))

DEFAULT_SEED = 2
set_all_seeds(DEFAULT_SEED, deterministic=True)

Ultralytics version: 8.3.0
Ultralytics checks.check_amp disabled to prevent YOLOv8n download.
Ultralytics torch_utils.check_amp disabled to prevent YOLOv8n download.
Ultralytics trainer-level AMP checks disabled.

--- Environment ---
{
  "timestamp": "2025-09-06T12:00:27.546343Z",
  "python": "3.10.18",
  "platform": "Windows-10-10.0.26100-SP0",
  "torch": "2.5.1",
  "cuda_available": true,
  "gpu": "NVIDIA GeForce RTX 3060 Ti"
}
Seeds set to 2. Deterministic: True


# Paths & Unique Run Directory

To ensure our experiments are organized and artifacts never overwrite each other, we will define a clear directory structure.

- **`PROJECT_ROOT`**: The main project folder.
- **`DATA_ROOT`**: The location of the dataset generated by `LabelConversion_VehicleOnly_Enhanced.py`.
- **`RUN_DIR`**: A **unique, timestamped directory** created for *this specific training run*. All outputs—checkpoints, logs, configs, and plots—will be saved here.

This practice is essential for MLOps, as it makes every run self-contained and easy to analyze or archive.

In [2]:
# Define Core Paths
PROJECT_ROOT = Path.cwd()
DATA_ROOT = PROJECT_ROOT / "mio_tcd_yolo_vehicles_only"
DATA_YAML = DATA_ROOT / "data.yaml"

# Paths to artifacts generated by the LabelConversion script
ARTIFACTS_PREP_DIR = DATA_ROOT / "artifacts"
HIERARCHY_JSON = ARTIFACTS_PREP_DIR / "hierarchy.json"
CLASS_WEIGHTS_JSON = ARTIFACTS_PREP_DIR / "class_weights.json"

# Create a Unique Directory for This Training Run
RUNS_ROOT = PROJECT_ROOT / "runs" / "yolo_l3_base"
RUN_ID = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
RUN_DIR = RUNS_ROOT / RUN_ID

# Create necessary subdirectories for organized output
ARTIFACTS_RUN_DIR = RUN_DIR / "artifacts"
CHECKPOINT_DIR = RUN_DIR / "checkpoints" # Ultralytics will write here automatically
for d in (RUN_DIR, ARTIFACTS_RUN_DIR):
    d.mkdir(parents=True, exist_ok=True)

# Save Environment Snapshot
with open(ARTIFACTS_RUN_DIR / "env.json", "w") as f:
    json.dump(ENV, f, indent=2)

print(f"PROJECT_ROOT: {PROJECT_ROOT}")
print(f"DATA_ROOT:    {DATA_ROOT}")
print(f"Unique Run Directory created at: {RUN_DIR}")

PROJECT_ROOT: c:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria
DATA_ROOT:    c:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria\mio_tcd_yolo_vehicles_only
Unique Run Directory created at: c:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria\runs\yolo_l3_base\20250906_120047


# Centralized Hyperparameter Configuration (hyp.yaml)

To ensure our runs are perfectly reproducible and easy to tune, we will define **all** training hyperparameters in a single YAML file. This file will be our "single source of truth."

**Advantages of this approach:**
- **Reproducibility:** Anyone can replicate this exact run using this notebook and the generated `hyp.yaml`.
- **Easy Tuning:** To change the learning rate or an augmentation, you only need to edit one value in one place.
- **Clarity:** It separates the *configuration* of the experiment from the *execution logic*.

The following hyperparameters are a strong starting point for **YOLO11**, optimized for a balance of speed and accuracy, leveraging state-of-the-art techniques like Focal Loss and Copy-Paste augmentation.

In [ ]:
# Define and Save Centralized Hyperparameter Configuration

# NOTE: These hyperparameters are a strong starting point for modern YOLO architectures.
# Tuned for YOLO11 and compatible with recent Ultralytics releases.
hyp_config = {
    # Optimizer & Scheduler
    'lr0': 0.01,          # Initial learning rate (for SGD, which 'auto' optimizer selects)
    'lrf': 0.01,          # Final learning rate multiplier (final_lr = lr0 * lrf)
    'momentum': 0.937,    # SGD momentum
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
    'cos_lr': True,       # Use cosine learning rate scheduler

    # Loss Components & Class Balancing
    'box': 7.5,           # Box loss gain
    'cls': 0.5,           # Classification loss gain (global multiplier for our per-class weights)
    'dfl': 1.5,           # Distribution Focal Loss gain
    #'fl_gamma': 1.5,      # Focal Loss gamma. > 0 turns on Focal Loss. 1.5 is a good start.

    # Augmentations (Optimized for vehicle detection)
    'mosaic': 1.0,        # Mosaic probability (powerful for object detection)
    'mixup': 0.1,         # Mixup probability
    'copy_paste': 0.1,    # Copy-paste augmentation gain. Excellent for rare classes.
    'degrees': 5.0,       # Image rotation (+/- deg)
    'translate': 0.1,     # Image translation (+/- fraction)
    'scale': 0.5,         # Image scale (+/- gain)
    'shear': 0.0,
    'perspective': 0.0,
    'flipud': 0.0,        # Flip up-down (usually not for vehicles)
    'fliplr': 0.5,        # Flip left-right
    'hsv_h': 0.015,       # Image HSV-Hue augmentation
    'hsv_s': 0.7,         # Image HSV-Saturation augmentation
    'hsv_v': 0.4,         # Image HSV-Value augmentation
}

# Save this config to the run's artifact directory for perfect reproducibility
HYP_YAML_PATH = ARTIFACTS_RUN_DIR / "hyp.yaml"
with open(HYP_YAML_PATH, 'w') as f:
    yaml.dump(hyp_config, f, sort_keys=False)

print(f"Hyperparameters centralized and saved to: {HYP_YAML_PATH}")
print("\n--- Hyperparameter Summary ---")
print(yaml.dump(hyp_config))

Hyperparameters centralized and saved to: c:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria\runs\yolo_l3_base\20250906_120047\artifacts\hyp.yaml

--- Hyperparameter Summary ---
box: 7.5
cls: 0.5
copy_paste: 0.1
cos_lr: true
degrees: 5.0
dfl: 1.5
fliplr: 0.5
flipud: 0.0
hsv_h: 0.015
hsv_s: 0.7
hsv_v: 0.4
lr0: 0.01
lrf: 0.01
mixup: 0.1
momentum: 0.937
mosaic: 1.0
perspective: 0.0
scale: 0.5
shear: 0.0
translate: 0.1
warmup_epochs: 3.0
warmup_momentum: 0.8
weight_decay: 0.0005



# Load Data Artifacts & Prepare Class Weights

This cell is responsible for loading the essential artifacts produced by the `LabelConversion_VehicleOnly_Enhanced.py` script.

**Workflow:**
1.  **Verify Paths:** It first checks that `data.yaml`, `hierarchy.json`, and `class_weights.json` exist, providing clear error messages if they are missing.
2.  **Load Class Counts:** It loads the pre-computed class instance counts for the leaf (L3) level from `class_weights.json`.
3.  **Load or Compute Weights:**
    *   It prioritizes loading the pre-calculated **effective-number (class-balanced)** weights, as these are generally superior for long-tailed distributions.
    *   If for some reason the pre-calculated weights are missing, it falls back to re-computing them on the fly using the loaded counts.
4.  **Prepare for Trainer:** The final weight vector (`CB_LEAF`) is prepared for injection into the Ultralytics trainer in a later step.

This defensive approach ensures our notebook can run even if some artifacts are missing, while always prioritizing the most robust pre-computed data.

In [5]:
# Load and Verify Data Artifacts, Prepare Class Weights

from collections import Counter

# Helper functions (mirroring the logic in the conversion script for consistency)
def effective_num_weights(counts, beta=0.9999, eps=1e-12):
    """Computes class-balanced weights based on the effective number of samples."""
    counts = np.array(counts, dtype=np.float64)
    eff = 1.0 - np.power(beta, counts)
    eff[eff < eps] = eps
    w = (1.0 - beta) / eff
    return (w / np.mean(w)).astype(np.float32)

# 1. Verify that essential files from the conversion script exist
essential_files = {
    "DATA_YAML": DATA_YAML,
    "HIERARCHY_JSON": HIERARCHY_JSON,
    "CLASS_WEIGHTS_JSON": CLASS_WEIGHTS_JSON,
}
for name, path in essential_files.items():
    if not path.exists():
        raise FileNotFoundError(f"Critical file missing: {name} at {path}. Please run LabelConversion_VehicleOnly_Enhanced.py first.")
    else:
        print(f"Found {name}: {path.name}")

# 2. Load the pre-computed class weights and names
with open(DATA_YAML, "r") as f:
    data_cfg = yaml.safe_load(f)
    class_names = data_cfg.get("names", [])
    if not class_names:
        raise ValueError("Class names are empty in data.yaml.")

with open(CLASS_WEIGHTS_JSON, "r") as f:
    weights_info = json.load(f)

try:
    leaf_level_info = weights_info["levels"]["leaf"]
    leaf_counts = np.array(leaf_level_info["counts"], dtype=np.int64)
    cb_leaf_precomputed = np.array(leaf_level_info["effective_num_weights"], dtype=np.float32)
    print("\nSuccessfully loaded pre-computed class counts and weights.")
except KeyError as e:
    print(f"Warning: Could not find pre-computed weights in {CLASS_WEIGHTS_JSON}. Will re-compute. Missing key: {e}")
    # This section is a fallback, but the script should have produced the weights.
    # You would need to add logic here to count from label files if this happens often.
    raise

# 3. Finalize the weight vector for this run
# We will use the pre-computed class-balanced (effective number) weights as our primary strategy.
CB_LEAF = cb_leaf_precomputed

# 4. Sanity Check and Summary Display
assert len(CB_LEAF) == len(class_names), f"Mismatch between number of weights ({len(CB_LEAF)}) and number of classes ({len(class_names)})."

print("\n--- Per-Class Counts & Effective-Number Weights (sorted by rarity) ---")
rows = sorted(zip(class_names, leaf_counts, CB_LEAF), key=lambda x: x[1]) # Sort by count
for name, count, weight in rows:
    print(f"  - {name:<20s} | Count: {count:>7,} | Weight: {weight:.4f}")

# 5. Persist the chosen weights to the run's artifact directory for record-keeping
run_weights_summary = {
    "class_names": class_names,
    "leaf_counts": leaf_counts.tolist(),
    "effective_number_weights_beta_0.9999": CB_LEAF.tolist(),
}
EXPERIMENT_WEIGHTS_PATH = ARTIFACTS_RUN_DIR / "experiment_class_weights.json"
with open(EXPERIMENT_WEIGHTS_PATH, "w") as f:
    json.dump(run_weights_summary, f, indent=2)
print(f"\nSaved a copy of the class weights for this run to: {EXPERIMENT_WEIGHTS_PATH.name}")

Found DATA_YAML: data.yaml
Found HIERARCHY_JSON: hierarchy.json
Found CLASS_WEIGHTS_JSON: class_weights.json

Successfully loaded pre-computed class counts and weights.

--- Per-Class Counts & Effective-Number Weights (sorted by rarity) ---
  - motorcycle           | Count:   1,273 | Weight: 2.2333
  - bicycle              | Count:   1,578 | Weight: 1.8287
  - single_unit_truck    | Count:   2,269 | Weight: 1.3150
  - work_van             | Count:   4,100 | Weight: 0.7937
  - articulated_truck    | Count:   4,830 | Weight: 0.6969
  - bus                  | Count:   6,152 | Weight: 0.5810
  - pickup_truck         | Count:  27,855 | Weight: 0.2845
  - car                  | Count: 140,485 | Weight: 0.2670

Saved a copy of the class weights for this run to: experiment_class_weights.json


# Efficient Smoke Test (Dry Run)

Before committing to a multi-hour training session, we must perform a quick "smoke test." This is a critical MLOps practice to catch configuration, data, or memory errors early.

**This test will:**
1.  Create a temporary `smoke_data.yaml` pointing to a very small subset of the training and validation data (e.g., 32 images).
2.  Initialize the **YOLO11n** model.
3.  Run `model.train()` for a single epoch on this tiny dataset.

**Expected Outcome:** The test should complete in **under a minute**. A successful run confirms that:
- The data paths are correct and images can be loaded.
- The model architecture is valid.
- The batch size fits into GPU memory.
- The training loop initializes without errors.

If this cell passes, we can proceed to the full training run with high confidence.

In [ ]:
'''# Efficient and Fast Smoke Test

from ultralytics import YOLO
import os

def create_smoke_test_yaml(data_yaml_path, run_dir, subset_size=32):
    """
    Creates a temporary data.yaml pointing to a small, valid subset of images
    for a fast and reliable smoke test.
    """
    with open(data_yaml_path, 'r') as f:
        data_cfg = yaml.safe_load(f)
    
    base_path = Path(data_yaml_path).parent
    smoke_data_cfg = data_cfg.copy()
    
    print(f"\n💨 Generating smoke test dataset with {subset_size} images per split...")
    
    for split in ['train', 'val']:
        original_img_dir = base_path / data_cfg[split]
        # Robustly find images, handling potential .txt files from ultralytics cache
        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
        all_images = sorted([p for p in original_img_dir.glob('*') if p.suffix.lower() in image_extensions])
        
        if not all_images:
            print(f"No images found in '{original_img_dir}' for smoke test. Skipping split.")
            continue
            
        subset_images = all_images[:subset_size]
        
        subset_file = run_dir / f"smoke_{split}.txt"
        with open(subset_file, 'w') as f:
            for img_path in subset_images:
                f.write(str(img_path.resolve()) + '\n')
        
        smoke_data_cfg[split] = str(subset_file.resolve())

    smoke_yaml_path = run_dir / "smoke_data.yaml"
    with open(smoke_yaml_path, 'w') as f:
        yaml.dump(smoke_data_cfg, f)
        
    print(f"   - Created smoke test config at: {smoke_yaml_path.name}")
    return smoke_yaml_path

# Run the Smoke Test
print("\nLaunching Smoke Test...")
try:
    # 1. Create the temporary data config
    smoke_yaml_path = create_smoke_test_yaml(DATA_YAML, ARTIFACTS_RUN_DIR)

    # 2. Initialize the YOLOv11n model. This will download the weights the first time.
    model = YOLO("yolo11n.pt")

    # 3. Run a minimal training loop (with W&B disabled for the smoke test)
    prev_wandb_disabled = os.environ.get("WANDB_DISABLED")
    os.environ["WANDB_DISABLED"] = "true"
    try:
        # We override key parameters for a fast, low-resource run.
        model.train(
            data=str(smoke_yaml_path),
            epochs=1,
            batch=4, # Small batch size to guarantee it fits in VRAM
            imgsz=320, # Small image size for speed
            workers=0, # Avoid dataloader overhead for tiny dataset
            cache=False, # No need to cache for one epoch
            project=RUN_DIR.name, # Use a subdirectory to keep logs separate
            name="smoke_test",
            verbose=False, # Keep the output clean
        )
    finally:
        # Restore previous WANDB_DISABLED state
        if prev_wandb_disabled is None:
            os.environ.pop("WANDB_DISABLED", None)
        else:
            os.environ["WANDB_DISABLED"] = prev_wandb_disabled
    
    # Clean up the .txt files but keep the yaml for inspection
    for split in ['train', 'val']:
        (ARTIFACTS_RUN_DIR / f"smoke_{split}.txt").unlink(missing_ok=True)

    print("\nSmoke test PASSED. The training pipeline is correctly configured for YOLOv11n.")

except Exception as e:
    print("\nSmoke test FAILED. Please check the error message below.")
    raise e

'''

'# Cell 10: Efficient and Fast Smoke Test\n\nfrom ultralytics import YOLO\nimport os\n\ndef create_smoke_test_yaml(data_yaml_path, run_dir, subset_size=32):\n    """\n    Creates a temporary data.yaml pointing to a small, valid subset of images\n    for a fast and reliable smoke test.\n    """\n    with open(data_yaml_path, \'r\') as f:\n        data_cfg = yaml.safe_load(f)\n    \n    base_path = Path(data_yaml_path).parent\n    smoke_data_cfg = data_cfg.copy()\n    \n    print(f"\n💨 Generating smoke test dataset with {subset_size} images per split...")\n    \n    for split in [\'train\', \'val\']:\n        original_img_dir = base_path / data_cfg[split]\n        # Robustly find images, handling potential .txt files from ultralytics cache\n        image_extensions = [\'.jpg\', \'.jpeg\', \'.png\', \'.bmp\']\n        all_images = sorted([p for p in original_img_dir.glob(\'*\') if p.suffix.lower() in image_extensions])\n        \n        if not all_images:\n            print(f"⚠️ No image

# Launching the Full Training Run

With all configurations prepared and the pipeline validated by the smoke test, we are ready to launch the full training run.

**This final step will:**
1.  Initialize the **YOLO11n** model with its official **COCO pre-trained weights**.
2.  Load our custom hyperparameters from the `hyp.yaml` file and pass them as individual arguments to the trainer.
3.  **Apply the effective-number class weights** directly to the classification loss function to combat class imbalance.
4.  Utilize hardware-specific optimizations (`cache=True`, `workers=8`, `amp=True`) to maximize training speed on your system.
5.  Run for the full `150` epochs, saving the best checkpoint based on validation mAP@50.
6.  Log all metrics, plots, and results to our unique `RUN_DIR` and, if enabled, to Weights & Biases.

In [ ]:
# The Full, Optimized Training Run (Corrected)

from ultralytics import YOLO
import torch

# 1. Initialize the YOLOv11n model with COCO pre-trained weights
model = YOLO("yolo11n.pt")

# 2. Load our centralized hyperparameters from the YAML file
with open(HYP_YAML_PATH, 'r') as f:
    hyp_config = yaml.safe_load(f)
print(f"Loaded hyperparameters from: {HYP_YAML_PATH.name}")

# 3. Prepare Class Weights for the Ultralytics Trainer
# The 'cls' gain in our YAML is a global multiplier. We apply it to our per-class weights.
cls_gain = hyp_config.get('cls', 0.5)
final_cls_weights = cls_gain * CB_LEAF  # CB_LEAF is the numpy array from Cell 8

# 4. Configure Hardware-Specific Optimizations
num_workers = 8

# 4.1 Auto-resume support (filter to YOLOv11 runs only)
RESUME_IF_AVAILABLE = True
resume_ckpt = None
if RESUME_IF_AVAILABLE:
    try:
        candidates = list((RUNS_ROOT).glob("*/weights/last.pt"))
        y11_candidates = []
        for ckpt in candidates:
            # ckpt path looks like: <...>/runs/yolo_l3_base/<RUN_ID>/weights/last.pt
            run_dir = ckpt.parents[1]  # the <RUN_ID> directory
            if run_dir.name == RUN_ID:
                # Skip the brand-new directory for this run
                continue
            args_yaml = run_dir / "args.yaml"
            if not args_yaml.exists():
                print(f"Skipping resume candidate without args.yaml: {run_dir}")
                continue
            try:
                args_text = args_yaml.read_text(encoding="utf-8", errors="ignore").lower()
            except Exception as _re:
                print(f"Skipping resume candidate (cannot read args.yaml): {run_dir} -> {_re}")
                continue
            if "yolo11" in args_text:
                y11_candidates.append(ckpt)
            else:
                print(f"Skipping non-YOLOv11 resume candidate: {run_dir}")
        if y11_candidates:
            resume_ckpt = max(y11_candidates, key=lambda p: p.stat().st_mtime)
            print(f"Found YOLOv11 checkpoint to resume: {resume_ckpt}")
    except Exception as _e:
        # Non-fatal; fall back to fresh training
        print(f"Resume scan error (ignored): {_e}")
        resume_ckpt = None

# 5. Assemble the Final Training Arguments
# This dictionary contains the core, non-hyperparameter settings.
base_args = {
    # Core Configuration
    "data": str(DATA_YAML),
    "epochs": 150,
    "batch": 32,
    "imgsz": 640,
    "optimizer": "auto",

    # Hardware & Performance
    "workers": num_workers,
    "cache": True,
    "amp": True,

    # Logging, Saving, & Project Management
    # Use only the folder name (e.g., 'yolo_l3_base') so Ultralytics saves to
    # runs/yolo_l3_base/RUN_ID locally while W&B gets a valid project name.
    "project": RUNS_ROOT.name,
    "name": RUN_ID,
    "save_period": 5,
    "patience": 20,
    
    # The crucial class weights argument
    "cls": final_cls_weights.tolist(),
}

# Apply resume option if available
if resume_ckpt:
    base_args["resume"] = str(resume_ckpt)
    print(f"   - Resuming training from: {resume_ckpt}")
else:
    base_args["resume"] = False

# MERGE ALL ARGUMENTS
# We unpack the loaded hyperparameters and the base arguments into one final dictionary.
# This is the modern, correct way to pass all settings.
training_args = {**base_args, **hyp_config}

# 6. (Optional) Login to Weights & Biases
try:
    if os.getenv("WANDB_API_KEY"):
        import wandb
        wandb.login()
        print("\nW&B login successful.")
except (ImportError, Exception):
    print("\nW&B not installed or login failed. Training will proceed with local logging.")

# 6.1 Safety: disable Ultralytics AMP probe (prevents yolov8n download)
try:
    from ultralytics.utils import torch_utils as _tu
    from ultralytics.utils import checks as _chk
    from ultralytics.engine import trainer as _tr
    _tu.check_amp = (lambda *a, **k: True)
    _chk.check_amp = (lambda *a, **k: True)
    if hasattr(_tr, 'check_amp'):
        _tr.check_amp = _tu.check_amp
    if hasattr(_tr, 'Trainer') and hasattr(_tr.Trainer, 'check_amp'):
        try:
            _tr.Trainer.check_amp = staticmethod(_tu.check_amp)
        except Exception:
            _tr.Trainer.check_amp = _tu.check_amp
    print("AMP probe disabled for training.")
except Exception as _e:
    print(f"Could not disable AMP probe in training cell: {_e}")

# 7. LAUNCH TRAINING
print("\nStarting full training run...")
print(f"   - All artifacts will be saved to: {RUN_DIR}")
print(f"   - Using {num_workers} dataloader workers.")
print(f"   - Class-balanced weights have been applied to the loss function.")

# Change into runs dir so Ultralytics creates the expected folder structure while
# keeping a W&B-safe project name.
_orig_cwd = os.getcwd()
os.chdir(PROJECT_ROOT / "runs")
try:
    results = model.train(**training_args)
finally:
    os.chdir(_orig_cwd)

# 8. Final Confirmation
print("\nTraining complete!")
print(f"   - Best model checkpoint saved at: {RUN_DIR / 'weights' / 'best.pt'}")
print(f"   - View detailed results and plots in the run directory: {RUN_DIR}")

100%|██████████| 5.35M/5.35M [00:00<00:00, 29.1MB/s]



✅ Loaded hyperparameters from: hyp.yaml
Skipping non-YOLOv11 resume candidate: c:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria\runs\yolo_l3_base\20250831_1135262
🔧 AMP probe disabled for training.

🚀 Starting full training run...
   - All artifacts will be saved to: c:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria\runs\yolo_l3_base\20250831_120130
   - Using 8 dataloader workers.
   - Class-balanced weights have been applied to the loss function.
New https://pypi.org/project/ultralytics/8.3.189 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.0  Python-3.10.18 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
Ultralytics 8.3.0  Python-3.10.18 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=c:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria\mio_tcd_yolo_vehicles_only\data.yaml, epochs=150, time=None, patience=20, batch=32, imgsz=640, save=True, save_period=5, cache=True, device=

wandb: Currently logged in as: mika47 (mika47-georg-august-uni) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Freezing layer 'model.23.dfl.conv.weight'


train: Scanning C:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria\mio_tcd_yolo_vehicles_only\labels\train.cache... 74746 images, 0 backgrounds, 0 corrupt: 100%|██████████| 74746/74746 [00:00<?, ?it/s]Scanning C:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria\mio_tcd_yolo_vehicles_only\labels\train.cache... 74746 images, 0 backgrounds, 0 corrupt: 100%|██████████| 74746/74746 [00:00<?, ?it/s]
train: Scanning C:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria\mio_tcd_yolo_vehicles_only\labels\train.cache... 74746 images, 0 backgrounds, 0 corrupt: 100%|██████████| 74746/74746 [00:00<?, ?it/s]Scanning C:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria\mio_tcd_yolo_vehicles_only\labels\train.cache... 74746 images, 0 backgrounds, 0 corrupt: 100%|██████████| 74746/74746 [00:00<?, ?it/s]


...

                   all      13190      48226      0.832      0.748       0.83      0.638

150 epochs completed in 20.827 hours.

150 epochs completed in 20.827 hours.
Optimizer stripped from yolo_l3_base\20250831_1201302\weights\last.pt, 5.5MB
Optimizer stripped from yolo_l3_base\20250831_1201302\weights\last.pt, 5.5MB
Optimizer stripped from yolo_l3_base\20250831_1201302\weights\best.pt, 5.5MB

Validating yolo_l3_base\20250831_1201302\weights\best.pt...
Ultralytics 8.3.0  Python-3.10.18 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
Optimizer stripped from yolo_l3_base\20250831_1201302\weights\best.pt, 5.5MB

Validating yolo_l3_base\20250831_1201302\weights\best.pt...
Ultralytics 8.3.0  Python-3.10.18 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
YOLO11n summary (fused): 238 layers, 2,583,712 parameters, 0 gradients, 6.3 GFLOPs
YOLO11n summary (fused): 238 layers, 2,583,712 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 207/207 [00:55<00:00,  3.71it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 207/207 [00:55<00:00,  3.71it/s]


                   all      13190      48226      0.828      0.751       0.83      0.638
                   bus       1641       1918      0.883      0.938      0.963      0.878
              work_van       2133       2344      0.823      0.526      0.697      0.549
     single_unit_truck       1771       1872      0.748      0.628      0.728      0.549
          pickup_truck       4382       6490      0.838      0.838      0.886      0.718
     articulated_truck       1693       1873       0.86      0.821      0.889      0.716
                   car      12563      33160      0.863      0.891      0.934      0.699
            motorcycle        258        277      0.894      0.671      0.781      0.494
               bicycle        237        292      0.712      0.692      0.762        0.5
                   bus       1641       1918      0.883      0.938      0.963      0.878
              work_van       2133       2344      0.823      0.526      0.697      0.549
     single_unit_truc

lr/pg0,███████▇▆▆▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
lr/pg1,▃▆██████▇▇▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁
lr/pg2,▃█████████▇▇▇▇▇▆▆▅▅▅▅▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁
metrics/mAP50(B),▁▃▅▆▆▇▇▇▇▇▇▇▇▇▇▇████████████████████████
metrics/mAP50-95(B),▁▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████████████
metrics/precision(B),▁▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▅▆▆▇▇▇▇▇▇▇▇▇███████████
metrics/recall(B),▁▃▂▄▇▇▇▇▇▇▇▇▇▇▇█▇▇██████████████████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▆▆▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁



🎉 Training complete!
   - Best model checkpoint saved at: c:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria\runs\yolo_l3_base\20250831_120130\weights\best.pt
   - View detailed results and plots in the run directory: c:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria\runs\yolo_l3_base\20250831_120130

   - Best model checkpoint saved at: c:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria\runs\yolo_l3_base\20250831_120130\weights\best.pt
   - View detailed results and plots in the run directory: c:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria\runs\yolo_l3_base\20250831_120130


# Post-Training Analysis & Next Steps

The training process is complete. The best model checkpoint, `best.pt`, has been saved based on its performance on the validation set.

**Analysis:**
- **Local Artifacts:** The run directory (`RUN_DIR`) contains all the outputs:
    - `weights/`: `best.pt` and `last.pt` checkpoints.
    - `results.csv`: A log of all metrics per epoch.
    - Confusion matrices, PR curves, and validation batch predictions.
- **Weights & Biases:** If enabled, your W&B dashboard provides an interactive way to analyze the run. Pay close attention to the **per-class AP plots** to see how well the model learned the rare classes.

**Next Steps:**
The `best.pt` model is now our foundational L3 detector. The next stage of the project is to use this model for:
1.  **Hard-Negative Mining:** Running inference on the dataset to find the exact locations where this model makes false-positive predictions.
2.  **Auxiliary Head Training:** Using the mined hard-negatives and true-positives to train our L1 and L2 heads, making them resilient to the specific failure modes of our L3 detector.

In [ ]:
# Final Validation on the Best Model

from ultralytics import YOLO

# Load the best performing model from the training run
try:
    best_model_path = Path("C:/Users/Mika/Desktop/New_Training_Run_Post_Bulgaria/runs/yolo_l3_base/20250831_1201302/weights/best.pt")
    if not best_model_path.exists():
        # Fallback for older ultralytics versions that might save in a different structure
        best_model_path = Path(results.save_dir) / 'weights' / 'best.pt'

    print(f"Loading best model from: {best_model_path}")
    model = YOLO(best_model_path)

    # Run validation to get the final, detailed metrics
    # This provides a clean, final report on the model's performance.
    print("\nRunning final validation on the best checkpoint...")
    final_metrics = model.val(
        data=str(DATA_YAML),
        imgsz=640,
        batch=32, # Use a smaller batch for validation to ensure no OOM errors
        workers=num_workers,
        split='val', # Explicitly run on the validation split
        plots=True, # Generate and save final plots
        project=RUN_DIR,
        name="final_validation"
    )

    print("\nFinal validation complete. Performance metrics are printed above.")
    print(f"   - Detailed validation results and plots saved in: {RUN_DIR / 'final_validation'}")

except Exception as e:
    print(f"\nCould not run final validation. Error: {e}")
    print("   - Please locate the 'best.pt' checkpoint in your run directory manually.")

Loading best model from: C:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria\runs\yolo_l3_base\20250831_1201302\weights\best.pt

🚀 Running final validation on the best checkpoint...
Ultralytics 8.3.0  Python-3.10.18 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
YOLO11n summary (fused): 238 layers, 2,583,712 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria\mio_tcd_yolo_vehicles_only\labels\val.cache... 13190 images, 0 backgrounds, 0 corrupt: 100%|██████████| 13190/13190 [00:00<?, ?it/s]Scanning C:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria\mio_tcd_yolo_vehicles_only\labels\val.cache... 13190 images, 0 backgrounds, 0 corrupt: 100%|██████████| 13190/13190 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 413/413 [01:04<00:00,  6.40it/s]


                   all      13190      48226      0.828      0.751       0.83      0.638
                   bus       1641       1918      0.884      0.938      0.963      0.878
              work_van       2133       2344      0.822      0.526      0.697      0.549
     single_unit_truck       1771       1872      0.749      0.628      0.728      0.549
          pickup_truck       4382       6490      0.839      0.838      0.886      0.719
     articulated_truck       1693       1873       0.86      0.821      0.889      0.715
                   car      12563      33160      0.864      0.891      0.934        0.7
            motorcycle        258        277      0.894      0.671      0.781      0.493
               bicycle        237        292       0.71      0.692      0.762      0.499
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to c:\Users\Mika\Desktop\New_Training_Run_Post_Bulgaria\runs\yolo_l3_base\20250831_120130\final_validat